# ING Den - final challenge
## Neuralna Ekipa

In this notebook we will do preliminary analysis of the dataset.

In [1]:
import pandas as pd

train_data = pd.read_csv('./datasets/in_time.csv')


In [2]:
import numpy as np

How much data did we recieve? 

In [3]:
train_data.shape

(310000, 307)

We recieved 310k observation with 307 features each. One feature of course is target binary variable, and the rest are features that are listed in Data_dictionary.xlsx. We built then the function, that splits data into specified categories and transform it into usable form. Then we fix -9999 values into NaNs.

In [4]:
def transform_data_analysis(X : pd.DataFrame):
    X = X.copy()
    X.set_index(['Customer_id'], inplace=True)
    real_variables_columns = pd.read_excel('Data_dictionary.xlsx').iloc[:42, :]
    types = {k:[] for k in real_variables_columns['Type'].unique()}
    X[X == -9999] = pd.NA
    real_variables_columns
    for feature in real_variables_columns.iterrows():
        # all variables with x on the end just land with 1-12
        if feature[1]['Column name*'] == 'Customer_id': continue
        if(feature[1]['Column name*'][-1] =='x'):
            for lag in range(13):
                types[feature[1]['Type']].append((feature[1]['Column name*'][:-1]+str(lag)).replace(' ', '_'))
        else:
            types[feature[1]['Type']].append(feature[1]['Column name*'].replace(' ', '_'))

    # at this time we include only numeric features
    return X.drop(['Target'], axis=1), X['Target']

Splitting data into exogenic and endogenic variable

In [5]:
from data_preparation.data_preparation import transform_data

X,y = transform_data(train_data)

Is there any missing target variable?

In [6]:
y.isna().any()

False

How many variables are missing?

In [7]:
np.mean(X.loc[:, (np.mean(X.isna(), axis=0) > 0).values].isna(), axis=0).sort_values(ascending=False)

Series([], dtype: float64)

We can see that 99,9532% of all external data is missing (credit card balance, term loan balance, mortgage balance) we will drop these features.

Then we see that also active_credit_lines has missing 88%, so we can create varaible *has_credit_card_lines* but drop this variable.

Then we see active_mortgages miss 82% of values, we will do the same as to the previous variable.

The two variables (and its' derevatives): limit_in_revolving_loans and utilized_limit_in_revolving_loans miss almost 20%, we will drop those features as it would be difficult to impute them.

Features to drop are below:

In [8]:
features_to_drop = np.mean(X.loc[:, (np.mean(X.isna(), axis=0) > 0).values].isna(), axis=0).index

In [9]:
features_to_drop

Index([], dtype='object')

but first we have to prepare some variables as specified above

In [10]:
def transform_data_analysis(X : pd.DataFrame):
    X = X.copy()
    X.set_index(['Customer_id'], inplace=True)
    real_variables_columns = pd.read_excel('Data_dictionary.xlsx').iloc[:42, :]
    types = {k:[] for k in real_variables_columns['Type'].unique()}
    X[X == -9999] = pd.NA
    real_variables_columns
    for feature in real_variables_columns.iterrows():
        # all variables with x on the end just land with 1-12
        if feature[1]['Column name*'] == 'Customer_id': continue
        if(feature[1]['Column name*'][-1] =='x'):
            for lag in range(13):
                types[feature[1]['Type']].append((feature[1]['Column name*'][:-1]+str(lag)).replace(' ', '_'))
        else:
            types[feature[1]['Type']].append(feature[1]['Column name*'].replace(' ', '_'))
    #features_to_drop = (X.loc[:, (np.mean(X.isna(), axis=0) > 0).values].isna()).any().index
    features_to_drop = ['Active_mortgages', 'Active_credit_card_lines',
       'External_term_loan_balance', 'External_mortgage_balance',
       'External_credit_card_balance', 'limit_in_revolving_loans_H12',
       'limit_in_revolving_loans_H11', 'limit_in_revolving_loans_H10',
       'limit_in_revolving_loans_H9', 'limit_in_revolving_loans_H8',
       'limit_in_revolving_loans_H7', 'limit_in_revolving_loans_H6',
       'limit_in_revolving_loans_H5', 'limit_in_revolving_loans_H4',
       'limit_in_revolving_loans_H3', 'limit_in_revolving_loans_H2',
       'limit_in_revolving_loans_H1', 'limit_in_revolving_loans_H0',
       'utilized_limit_in_revolving_loans_H12',
       'utilized_limit_in_revolving_loans_H11',
       'utilized_limit_in_revolving_loans_H10',
       'utilized_limit_in_revolving_loans_H9',
       'utilized_limit_in_revolving_loans_H8',
       'utilized_limit_in_revolving_loans_H7',
       'utilized_limit_in_revolving_loans_H6',
       'utilized_limit_in_revolving_loans_H5',
       'utilized_limit_in_revolving_loans_H4',
       'utilized_limit_in_revolving_loans_H3',
       'utilized_limit_in_revolving_loans_H2',
       'utilized_limit_in_revolving_loans_H1',
       'utilized_limit_in_revolving_loans_H0']
    types['Created'] = []
    
    # create features that need missing values
    types['Created'].append('hasExternal_credit_card_balance')
    types['Created'].append('hasExternal_term_loan_balance')
    types['Created'].append('hasExternal_mortgage_balance')
    types['Created'].append('hasActive_credit_card_lines')
    types['Created'].append('hasActive_mortgages')

    X['hasExternal_credit_card_balance'] = ~pd.isna(X['External_credit_card_balance'])
    X['hasExternal_term_loan_balance'] = ~pd.isna(X['External_term_loan_balance'])
    X['hasExternal_mortgage_balance'] = ~pd.isna(X['External_mortgage_balance'])
    X['hasActive_credit_card_lines'] = ~pd.isna(X['Active_credit_card_lines'])
    X['hasActive_mortgages'] = ~pd.isna(X['Active_mortgages'])

    # here we drop features that are missing, at this point we have 
    X = X.drop(features_to_drop, axis=1)

    create_new_features(X)
    
    return X.drop(['Target'] + types['MM-YYYY'] + types['DD-MM-YYYY'], axis=1), X['Target']

Are there any missing:

In [13]:
(transform_data_analysis(train_data)[0].isna().any() >0).any()

False

## Feature engineering

In this section we will create new variables by hand. At this stage of pipeline, there should be no missing features in the dataset.

In [12]:
def create_new_features(X : pd.DataFrame):
    # this function will (in a sense) create new features at the stage where we have no missing values
    # quartile sum of all incoming transactions
    for idx, quarter in enumerate([0, 3, 6, 9]):
        X[f"inc_transactions_Quartile{idx+1}"] = X[f"inc_transactions_H{quarter}"] + X[f"inc_transactions_H{quarter+1}"] + X[f"inc_transactions_H{quarter+2}"]

    # quratile sum of all outcoming transactions
    for idx, quarter in enumerate([0, 3, 6, 9]):
        X[f"out_transactions_Quartile{idx+1}"] = X[f"out_transactions_H{quarter}"] + X[f"out_transactions_H{quarter+1}"] + X[f"out_transactions_H{quarter+2}"]

    #quartile sum of values of incoming transactions
    for idx, quarter in enumerate([0, 3, 6, 9]):
        X[f"inc_transactions_amt_Quartile{idx+1}"] = X[f"inc_transactions_amt_H{quarter}"] + X[f"inc_transactions_amt_H{quarter+1}"] + X[f"inc_transactions_amt_H{quarter+2}"]

    #quartile sum of values of outcoming transactions
    for idx, quarter in enumerate([0, 3, 6, 9]):
        X[f"out_transactions_amt_Quartile{idx+1}"] = X[f"out_transactions_amt_H{quarter}"] + X[f"out_transactions_amt_H{quarter+1}"] + X[f"out_transactions_amt_H{quarter+2}"]

    #quartile sum of os_term_loan
    for idx, quarter in enumerate([0, 3, 6, 9]):
        X[f"Os_term_loan_Quartile{idx+1}"] = X[f"Os_term_loan_H{quarter}"] + X[f"Os_term_loan_H{quarter+1}"] + X[f"Os_term_loan_H{quarter+2}"]

    #quartile sum of os_credit_card
    for idx, quarter in enumerate([0, 3, 6, 9]):
        X[f"Os_credit_card_Quartile{idx+1}"] = X[f"Os_credit_card_H{quarter}"] + X[f"Os_term_loan_H{quarter+1}"] + X[f"Os_term_loan_H{quarter+2}"]

    #quartile sum of os_mortgage
    for idx, quarter in enumerate([0, 3, 6, 9]):
        X[f"Os_mortgage_Quartile{idx+1}"] = X[f"Os_mortgage_H{quarter}"] + X[f"Os_mortgage_H{quarter+1}"] + X[f"Os_mortgage_H{quarter+2}"]

    #time in current job / time in address
    X["TimeInJobPerTimeInAddress"] = X["Time_in_current_job"]/X["Time_in_address"]

    #percent of incomes that go to current_acount
    for month in range(0, 13):
        X[f"incPerCurrentAccountBalance{month}"] = X[f"inc_transactions_H{month}"]/X[f"Current_amount_balance_H{month}"]
        
    #percent of incomes that go to savings_account
    for month in range(0, 13):
        X[f"incPerSavingsAccountBalance{month}"] = X[f"inc_transactions_H{month}"]/X[f"Savings_amount_balance_H{month}"]


In [14]:
transform_data_analysis(train_data)

(             No_dependants  Time_in_address  Time_in_current_job  \
 Customer_id                                                        
 33344475                 0             3.41                 4.13   
 38290809                 1             2.28                11.00   
 33178125                 1             8.99                 1.73   
 37129367                 2             5.55                 1.76   
 31904710                 1             4.48                 8.43   
 ...                    ...              ...                  ...   
 31104928                 1             1.92                 0.63   
 38699158                 1            19.89                 3.03   
 37435274                 3             3.92                 3.90   
 36145194                 1            18.38                14.47   
 37156247                 1             8.04                 0.60   
 
              Credit_cards  Debit_cards  Active_accounts  Active_loans  \
 Customer_id               